# Homework 3 (Problem 8, 2pts)
## ME 334: Advanced Dynamics, Control and System Identification
### Stanford University

In [ ]:
# imports
import numpy as np
import sympy as sp
from sympy import *
from sympy.vector import *
init_printing(use_unicode=True)

## Setup

![image.png](attachment:image.png)

In [ ]:
# defining free parameters
th1 = symbols(r'\theta_1')  # variables
th2 = symbols(r'\theta_2')
r3 = symbols(r'r_3')
dth1 = symbols(r'\dot{\theta}_1')  # first derivatives
dth2 = symbols(r'\dot{\theta}_2')
dr3 = symbols(r'\dot{r}_3')
ddth1 = symbols(r'\ddot{\theta}_1')  # second derivatives
ddth2 = symbols(r'\ddot{\theta}_2')
ddr3 = symbols(r'\ddot{r}_3')

In [ ]:
## helper functions for kinematics
def rot_z(theta):
    R = Matrix([
        [sp.cos(theta), -sp.sin(theta), 0],
        [sp.sin(theta), sp.cos(theta), 0],
        [0, 0, 1],
    ])
    return R

def rot_y(theta):
    R = Matrix([
        [sp.cos(theta), 0, sp.sin(theta)],
        [0,1,0],
        [-sp.sin(theta),0, sp.cos(theta)],
    ])
    return R

def rot_x(theta):
    R = Matrix([
        [1, 0, 0],
        [0, sp.cos(theta), -sp.sin(theta)],
        [0, sp.sin(theta), sp.cos(theta)],
    ])
    return R

In [ ]:
## frames and joint variables

## [TODO] Define rotations between frames (you can use the above functions)

R_AB = Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
R_BC = Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [ ]:
## link geometry 
########### (WATCH OUT FOR UNITS!) #############

## [TODO] Fill in the data from the problem

# link 1
H1v = 0 #height of vertical cylinder
R1v = 0 #radius of vertical cylinder
H1h = 0 #height (length) of horizontal cylinder
R1h = 0 #radius of horizontal cylinder

# link 2
H2 = 0 
R2i = 0
R2e = 0

# link 3
H3 = 0
R3 = 0

## density of aluminum
rho = 0

## kinematic params
L1 = 0 #OQ
L2 = 0 #QR

First, we compute the locations of the centers of mass of each link.

Second, we compute the velocities of the centers of mass of each link.

At the given state, the velocities are:

In [ ]:
# You may use this substitution rule (lengths are in meters here, change if you're doing everything in mm!): 
# subs([
#    (th1, -sqrt(2)),
#    (th2, sqrt(3)),
#    (r3, sqrt(5)/10),
#    (dth1, sqrt(7)),
#    (dth2, -sqrt(11)),
#    (dr3, sqrt(13)/1000),
# ]) 

## Use N() to report values in decimal form
## Specify units of your final answer

Third, we express the angular velocities of each link in convenient frames for the energy calculation.

At the given state, the angular velocities are:

In [ ]:
# You may use this substitution rule (lengths are in meters here, change if you're doing everything in mm!): 
# subs([
#    (th1, -sqrt(2)),
#    (th2, sqrt(3)),
#    (r3, sqrt(5)/10),
#    (dth1, sqrt(7)),
#    (dth2, -sqrt(11)),
#    (dr3, sqrt(13)/1000),
# ]) 

## Use N() to report values in decimal form
## Specify units of your final answer

Fourth, we compute the inertia tensors of each link. Following the integration procedure here: https://scienceworld.wolfram.com/physics/MomentofInertiaCylinder.html, we know that the inertia tensor of a solid cylinder is

In [ ]:
# helper functions for inertia calculation

def Ic_cylinder(R, H, rho):
    Ic = (np.pi * rho * R ** 2 * H) * Matrix([
        [(H ** 2) / 12 + (R ** 2) / 4, 0, 0], 
        [0, (H ** 2) / 12 + (R ** 2) / 4, 0],
        [0, 0, (R ** 2) / 2],
    ])
    return Ic

def inertia_particle(p1, p2):
    x = p1[0] - p2[0]
    y = p1[1] - p2[1]
    z = p1[2] - p2[2]
    I = Matrix([
        [y ** 2 + z ** 2, -x * y, -x * z],
        [-x * y, x ** 2 + z ** 2, -y * z],
        [-x * z, -y * z, x ** 2 + y ** 2],
    ])
    return I

Finally, we compute the total kinetic energy of the system. 

Note that inertias and angular velocities can be expressed in any SRT, but the inertia tensor for any particular link and its angular velocity have to expressed in the same SRT for the energy term to make sense.

At the given state, in Nm, the kinetic energy is:



In [ ]:
# You may use this substitution rule (lengths are in meters here, change if you're doing everything in mm!): 
# subs([
#    (th1, -sqrt(2)),
#    (th2, sqrt(3)),
#    (r3, sqrt(5)/10),
#    (dth1, sqrt(7)),
#    (dth2, -sqrt(11)),
#    (dr3, sqrt(13)/1000),
# ]) 

## Use N() to report values in decimal form
## Specify units of your final answer